<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/chroma_auto_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-Retrieval from a Weaviate Vector Database

This guide shows how to perform **auto-retrieval** in LlamaIndex with [Weaviate](https://weaviate.io/). 

The Weaviate vector database supports a set of [metadata filters](https://weaviate.io/developers/weaviate/search/filters) in addition to a query string for semantic search. Given a natural language query, we first use a Large Language Model (LLM) to infer a set of metadata filters as well as the right query string to pass to the vector database (either can also be blank). This overall query bundle is then executed against the vector database.

This allows for more dynamic, expressive forms of retrieval beyond top-k semantic search. The relevant context for a given query may only require filtering on a metadata tag, or require a joint combination of filtering + semantic search within the filtered set, or just raw semantic search.

## Setup 

We first define imports and define an empty Weaviate collection.

If you're opening this Notebook on Colab, you will probably need to install LlamaIndex 🦙.

In [15]:
%pip install llama-index-vector-stores-weaviate

Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install llama-index weaviate-client

In [17]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

We will be using GPT-4 for its reasoning capabilities to infer the metadata filters. Depending on your use case, `"gpt-3.5-turbo"` can work as well.

In [18]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [19]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

Settings.llm = OpenAI(model="gpt-4")
Settings.embed_model = OpenAIEmbedding()

This Notebook uses Weaviate in [Embedded mode](https://weaviate.io/developers/weaviate/installation/embedded), which is supported on Linux and macOS.

If you prefer to try out Weaviate's fully managed service, [Weaviate Cloud Services (WCS)](https://weaviate.io/developers/weaviate/installation/weaviate-cloud-services), you can enable the code in the comments.

In [20]:
import weaviate
from weaviate.embedded import EmbeddedOptions

# Connect to Weaviate client in embedded mode
client = weaviate.Client(embedded_options=EmbeddedOptions())

# Enable this code if you want to use Weaviate Cloud Services instead of Embedded mode.
"""
import weaviate

# cloud
resource_owner_config = weaviate.AuthClientPassword(
    username="",
    password="",
)
client = weaviate.Client(
    "https://test.weaviate.network",
    auth_client_secret=resource_owner_config,
)

# local
# client = weaviate.Client("http://localhost:8081")
"""


embedded weaviate is already listening on port 8079


/opt/homebrew/lib/python3.11/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.2.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


'\nimport weaviate\n\n# cloud\nresource_owner_config = weaviate.AuthClientPassword(\n    username="",\n    password="",\n)\nclient = weaviate.Client(\n    "https://test.weaviate.network",\n    auth_client_secret=resource_owner_config,\n)\n\n# local\n# client = weaviate.Client("http://localhost:8081")\n'

## Defining Some Sample Data

We insert some sample nodes containing text chunks into the vector database. Note that each `TextNode` not only contains the text, but also metadata e.g. `category` and `country`. These metadata fields will get converted/stored as such in the underlying vector db.

In [21]:
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text=(
            "Michael Jordan is a retired professional basketball player,"
            " widely regarded as one of the greatest basketball players of all"
            " time."
        ),
        metadata={
            "category": "Sports",
            "country": "United States",
        },
    ),
    TextNode(
        text=(
            "Angelina Jolie is an American actress, filmmaker, and"
            " humanitarian. She has received numerous awards for her acting"
            " and is known for her philanthropic work."
        ),
        metadata={
            "category": "Entertainment",
            "country": "United States",
        },
    ),
    TextNode(
        text=(
            "Elon Musk is a business magnate, industrial designer, and"
            " engineer. He is the founder, CEO, and lead designer of SpaceX,"
            " Tesla, Inc., Neuralink, and The Boring Company."
        ),
        metadata={
            "category": "Business",
            "country": "United States",
        },
    ),
    TextNode(
        text=(
            "Rihanna is a Barbadian singer, actress, and businesswoman. She"
            " has achieved significant success in the music industry and is"
            " known for her versatile musical style."
        ),
        metadata={
            "category": "Music",
            "country": "Barbados",
        },
    ),
    TextNode(
        text=(
            "Cristiano Ronaldo is a Portuguese professional footballer who is"
            " considered one of the greatest football players of all time. He"
            " has won numerous awards and set multiple records during his"
            " career."
        ),
        metadata={
            "category": "Sports",
            "country": "Portugal",
        },
    ),
]

## Build Vector Index with Weaviate Vector Store

Here we load the data into the vector store. As mentioned above, both the text and metadata for each node will get converted into corresopnding representations in Weaviate. We can now run semantic queries and also metadata filtering on this data from Weaviate.

In [22]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore

vector_store = WeaviateVectorStore(
    weaviate_client=client, 
    index_name="LlamaIndex_filter"
)
                                   
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [23]:
index = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Define `VectorIndexAutoRetriever`

We define our core `VectorIndexAutoRetriever` module. The module takes in `VectorStoreInfo`,
which contains a structured description of the vector store collection and the metadata filters it supports.
This information will then be used in the auto-retrieval prompt where the LLM infers metadata filters.

In [24]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info="brief biography of celebrities",
    metadata_info=[
        MetadataInfo(
            name="category",
            type="str",
            description=(
                "Category of the celebrity, one of [Sports, Entertainment,"
                " Business, Music]"
            ),
        ),
        MetadataInfo(
            name="country",
            type="str",
            description=(
                "Country of the celebrity, one of [United States, Barbados,"
                " Portugal]"
            ),
        ),
    ],
)

retriever = VectorIndexAutoRetriever(
    index, vector_store_info=vector_store_info
)

## Running over some sample data

We try running over some sample data. Note how metadata filters are inferred - this helps with more precise retrieval! 

In [25]:
response = retriever.retrieve("Tell me about two celebrities from United States")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Tell me about celebrities
Using query str: Tell me about celebrities
Using query str: Tell me about celebrities
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('country', '==', 'United States')]
Using filters: [('country', '==', 'United States')]
Using filters: [('country', '==', 'United States')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Req

In [26]:
print(response)

print("Text:")
print(response[0].text)

[NodeWithScore(node=TextNode(id_='a6ba3fa4-3ced-4281-9cf3-df9e73bb92d2', embedding=[-0.016137771, -0.01626444, -0.010450283, -0.015225747, -0.0023813979, 0.01203366, -0.019013183, 0.0016736287, -0.03131285, -0.02898212, 0.01707513, 0.0151117435, -0.004363785, 0.0053074774, -0.017024461, 0.003290256, 0.03143952, 0.018265828, 0.005095305, -0.02923546, -0.024789339, 0.023294631, -0.0028247433, -0.00044255366, -0.009018911, 0.0067641833, 0.0123883365, -0.025245352, 0.01508641, -0.030350156, -0.032807555, -0.007334199, 0.0032585885, -0.030831503, -0.017556475, 0.013908377, -0.0015556673, -0.03409959, 0.018189827, 0.0027661584, 0.027892757, 0.0015176662, -0.007897881, -0.02410532, -0.014731733, 0.008581899, 0.0068275183, -0.036075644, -0.013566368, 0.013731039, 0.014972406, 0.016973794, -0.018443167, -0.018671174, -0.028627444, -0.002777242, -0.016733121, -0.004585458, 0.02779142, -0.017721148, 0.0035530964, 0.021584585, -0.023788646, -0.017100465, -0.031920865, -0.018873846, -0.03181953, 0.

In [27]:
response = retriever.retrieve("Tell me about Sports celebrities from United States")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Sports celebrities
Using query str: Sports celebrities
Using query str: Sports celebrities
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('category', '==', 'Sports'), ('country', '==', 'United States')]
Using filters: [('category', '==', 'Sports'), ('country', '==', 'United States')]
Using filters: [('category', '==', 'Sports'), ('country', '==', 'United States')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request:

In [28]:
print(response)

print("Text:")
print(response[0].text)

[NodeWithScore(node=TextNode(id_='5e689253-2e2c-440d-9f72-6f9513fd2c3a', embedding=[-0.018109553, -0.033595245, -0.019237569, -0.0061059967, 0.003987903, 0.024718253, -0.029107705, -0.021346467, -0.02611601, -0.025821745, 0.009428738, -0.006639352, 0.0008498432, -0.011415026, 0.0058025364, -0.013879495, 0.013107049, -0.016638229, 0.015755434, -0.0112249805, -0.004300559, 0.02291588, -0.0029824974, -0.024019374, 0.0017288063, 0.0142228035, 0.005716709, -0.008202635, 0.025355827, -0.023013968, -0.00620102, -0.005863841, -0.017668156, -0.031093992, -0.02103994, 0.00898121, -0.013523925, -0.01770494, 0.02502478, -0.0014559983, 0.021959519, 0.02314884, -0.0196054, -0.01891878, 0.007344362, -0.0035526357, 0.017803026, -0.038426094, 0.003091314, 0.019114958, 0.010403491, 0.015534735, -0.004855371, -0.043036245, 0.010924585, -0.016233614, -0.02303849, 0.0067558317, 0.014590635, -0.028053254, 0.0017471979, 0.013290965, -0.025968878, 0.006541264, 0.0087114675, 0.0033963076, -0.015081077, -0.0107